In [1]:
# !pip install -q git+https://github.com/tcstrength/item-identification.git@main timm

In [2]:
import os
import pandas as pd
from pathlib import Path
from hcmus.core import appconfig
from hcmus.lbs import LabelStudioConnector
from hcmus.utils import viz_utils

2025-06-11 13:56:38.826 | INFO     | hcmus.core.appconfig:<module>:7 - Load DotEnv: True
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
accepted_labels = """
"""
accepted_labels = accepted_labels.splitlines()

In [4]:
splits = {}
for split_name in ["train", "test", "val"]:
    lsb_connector = LabelStudioConnector(
        url=appconfig.LABEL_STUDIO_URL,
        api_key=appconfig.LABEL_STUDIO_API_KEY,
        project_id=appconfig.LABEL_STUDIO_PROJECT_MAPPING[split_name],
        temp_dir=appconfig.LABEL_STUDIO_TEMP_DIR
    )

    tasks = lsb_connector.get_tasks()
    labels = lsb_connector.extract_labels(tasks)
    dataset = lsb_connector.download_dataset(tasks, labels)
    dataset = [x for x in dataset if x.get("target").get("labels")]
    idx2label = {v: k for k, v in labels.items()}
    for item in dataset:
        new_labels = []
        for idx in item.get("target").get("labels"):
            label_str = idx2label[idx]
            if label_str in accepted_labels:
                new_labels.append(idx2label[idx])
            else:
                # Compatible with SKU110k
                new_labels.append("object")
        item.get("target")["labels"] = new_labels
    splits[split_name] = dataset

2025-06-11 13:56:40.591 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:122 - Page size is too large, only 3325 tasks available.
2025-06-11 13:56:40.591 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:126 - New `page_to` applied: 34
Loading tasks: 100%|██████████| 34/34 [00:09<00:00,  3.57it/s]
2025-06-11 13:59:05.693 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:122 - Page size is too large, only 378 tasks available.
2025-06-11 13:59:05.694 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:126 - New `page_to` applied: 4
Loading tasks: 100%|██████████| 4/4 [00:03<00:00,  1.26it/s]
2025-06-11 13:59:19.826 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:122 - Page size is too large, only 30 tasks available.
2025-06-11 13:59:19.827 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:126 - New `page_to` applied: 1
Loading tasks: 100%|██████████| 1/1 [00:00<00:00,  6.00it/s]


In [5]:
import os
import json
import shutil
from PIL import Image

def generate_categories(splits):
    label_set = set()
    for split_data in splits.values():
        for item in split_data:
            label_set.update(item["target"]["labels"])
    label_list = sorted(label_set)
    return [{"id": i + 1, "name": label} for i, label in enumerate(label_list)], {
        label: i + 1 for i, label in enumerate(label_list)
    }

def convert_split_to_coco(data, categories_dict, split_name, split_output_dir):
    images_dir = os.path.join(split_output_dir, "images")
    os.makedirs(images_dir, exist_ok=True)

    coco_dict = {
        "images": [],
        "annotations": [],
        "categories": [{"id": cid, "name": name} for name, cid in categories_dict.items()]
    }

    ann_id = 1
    for img_id, item in enumerate(data):
        img_path = item["image"]
        target = item["target"]
        boxes = target["boxes"]
        labels = target["labels"]

        file_name = os.path.basename(img_path)
        dst_path = os.path.join(images_dir, file_name)
        shutil.copyfile(img_path, dst_path)

        with Image.open(img_path) as img:
            width, height = img.size

        coco_dict["images"].append({
            "id": img_id,
            "file_name": file_name,
            "width": width,
            "height": height
        })

        for box, label in zip(boxes, labels):
            x1, y1, x2, y2 = box
            bbox = [x1, y1, x2 - x1, y2 - y1]
            area = bbox[2] * bbox[3]

            coco_dict["annotations"].append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": categories_dict[label],
                "bbox": bbox,
                "area": area,
                "iscrowd": 0
            })
            ann_id += 1

    # Save JSON in split folder
    json_path = os.path.join(split_output_dir, f"annotations_{split_name}.json")
    with open(json_path, "w") as f:
        json.dump(coco_dict, f, indent=2)

    print(f"✔ {split_name}: saved {len(data)} images and annotations to {split_output_dir}")

def convert_splits_to_coco(splits, base_output_dir):
    os.makedirs(base_output_dir, exist_ok=True)

    _, label_to_id = generate_categories(splits)

    for split_name, data in splits.items():
        split_output_dir = os.path.join(base_output_dir, split_name)
        os.makedirs(split_output_dir, exist_ok=True)
        convert_split_to_coco(data, label_to_id, split_name, split_output_dir)


In [ ]:
convert_splits_to_coco(splits, "/Volumes/Cucumber/Projects/datasets/curated/hcmus-iid-object")

✔ train: saved 3071 images and annotations to /Volumes/Cucumber/Projects/datasets/hcmus-iid-object/train
✔ test: saved 346 images and annotations to /Volumes/Cucumber/Projects/datasets/hcmus-iid-object/test
✔ val: saved 30 images and annotations to /Volumes/Cucumber/Projects/datasets/hcmus-iid-object/val
